In [1]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# # open it, go to a website, and get results
driver = webdriver.Chrome('chromedriver',options=options)

In [2]:
from datetime import date, timedelta
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
#from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import time

In [3]:
url = "https://www.gismeteo.ru/diary/4368/"
driver.get(url)

In [4]:
def getMonthString(n): #from number get string name of month
    if n == 1:
        return "Январь"
    elif n == 2:
        return "Февраль"
    elif n == 3:
        return "Март"
    elif n == 4:
        return "Апрель"
    elif n == 5:
        return "Май"
    elif n == 6:
        return "Июнь"
    elif n == 7:
        return "Июль"
    elif n == 8:
        return "Август"
    elif n == 9:
        return "Сентябрь"
    elif n == 10:
        return "Октябрь"
    elif n == 11:
        return "Ноябрь"
    elif n == 12:
        return "Декабрь"

In [5]:
def findValues(country, city_id, city,year, month, date_vals, day_temp_vals, evening_temp_vals, city_ids): #choose country, city and date to get temperature values
    #year_select = driver.find_element(By.NAME, "Year") 
    #year_options = year_select.find_elements(By.TAG_NAME, 'option')
    #Select year_option = new Select(year_select);
    #year_option.selectByVisibleText(str(year));
    
    
    
    select = driver.find_element(By.NAME, "sd_country")
    options = select.find_elements(By.TAG_NAME, 'option')
    for option in options:
        if option.text == country:
            print("Set country = ", option.text)
            option.click()
    
    select = driver.find_element(By.NAME, "sd_distr")
    options = select.find_elements(By.TAG_NAME, 'option')
    region = "Все области"
    for option in options:
        if option.text == str(region):
            print("Set region = ", option.text)
            option.click()
            break
    time.sleep(10)
    select = driver.find_element(By.NAME, "sd_city")
    options = select.find_elements(By.TAG_NAME, 'option')
    #print("City options ", options)
    for option in options:
        print("Try to set city ", option.text)
        if option.text == str(city):
            print("Set city = ", option.text)
            option.click()
            break
    
    
    print("Set year = ", str(year))
    year_select = Select(WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "date_Year"))))
    year_select.select_by_visible_text(str(year))

    #field = driver.find_element(By.NAME, "Year") 
    #field.send_keys(str(year))
    
    #for year_option in year_options:
    #    if year_option.text == str(year):
    #        print("Set year = ", year_option.text)
    #        year_option.click()
    #month_select = driver.find_element(By.NAME, "Month")
    #month_options = month_select.find_elements(By.TAG_NAME, 'option')
    #Select month_option = new Select(month_select);
    #month_option.selectByVisibleText(getMonthString(month));
    print("Set month = ", getMonthString(month))
    month_select = Select(WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "date_Month"))))
    month_select.select_by_visible_text(getMonthString(month))
    button = driver.find_element(By.ID, "selector_go_btn")
    #print("Buton ", button)
    button.click()
    #field = driver.find_element(By.NAME, "Month") 
    #field.send_keys(getMonthString(month))
    

    #for month_option in month_options:
        
        #if month_option.text == getMonthString(month):
        #    print("Set month = ", month_option.text)
        #    Select(month_option)
        #    month_option.click()
    days = driver.find_elements(By.CLASS_NAME, "first")
    temps = driver.find_elements(By.CLASS_NAME, "first_in_group")
    for d in days:
        if d.text != "Число":
            #print("day = ", d.text)
            new_date = "" + str(year) + "/" + str(month) + "/" + d.text;
            city_ids.append(city_id)
            date_vals.append(new_date)
    counter = 0
    for t in temps:
        
        if t.text != "День" and t.text != "Вечер" and t.text !="Температура"and t.text !="Число":
            
            if counter % 2 == 0:
                #print("Day temp = ", t.text)
                day_temp_vals.append(t.text)
            else:
                #print("Evening temp = ", t.text)
                evening_temp_vals.append(t.text)
            counter += 1

In [6]:
def getCities(country): #get all cities (centers of regions)
    #ids = []
    names = []
    #country_select = Select(WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "sd_country"))))
    #country_select.select_by_visible_text(country)
    time.sleep(10)
    
    #print(driver.find_element(By.TAG_NAME, "html").text)
    select = driver.find_element(By.NAME, "sd_country")
    options = select.find_elements(By.TAG_NAME, 'option')
    for option in options:
        if option.text == str(country):
            option.click()
            break
    select = driver.find_element(By.NAME, "sd_distr")
    options = select.find_elements(By.TAG_NAME, 'option')
    region = "Все области"
    for option in options:
        if option.text == str(region):
            option.click()
            break
    #counter = 0
    select = driver.find_element(By.NAME, "sd_city")
    options = select.find_elements(By.TAG_NAME, 'option')
    for option in options:
        #ids.append(counter)
        #counter += 1
        names.append(option.text)

    
        
    df = pd.DataFrame({ #'Id': np.array(ids),
                         'Name': np.array(names)})
    
    return df

In [36]:
def getWeather(country, city, city_id, counter, main_city): #parse all values 
    titles = []
    #url = "https://www.gismeteo.ru/diary/4368/"
    #driver.get(url)
    #print("url = ", url)
    date_vals = []
    day_temp_vals = []
    evening_temp_vals = []
    city_ids = []
    today = date.today()
    
    
    today_year = today.year
    today_month = today.month
    print("today_year = ", today_year)
    print("today_month = ", today_month)
    if str(city) == str(main_city):
        print("Check Уфа")
        first_date = date.today() - timedelta(days=365)
        first_year = first_date.year
        number_of_years = today_year - first_year
        print("First date = ", first_date)
        print("Current date = ", today)
            #смотрим 2 года
        if number_of_years == 1:
            print("2 years")
            for year in [today_year, first_year]:
                if year == first_year:
                    months = list(range(today_month, 13))
                    for month in months:
                        findValues(country, city_id, city, year, month, date_vals, day_temp_vals, evening_temp_vals, city_ids)
                elif year == today_year:
                    months = list(range(1, today_month + 1))
                    for month in months:
                        findValues(country, city_id, city,year, month, date_vals, day_temp_vals, evening_temp_vals, city_ids)    

          #смотрим 1 год текущий
        elif number_of_years == 0:
            for year in [today_year]:
                months = list(range(1, today_month + 1))
                for month in months:
                    findValues(country, city_id, city,year, month, date_vals, day_temp_vals, evening_temp_vals, city_ids) 
 
                            
    else: #смотрим 2 последних месяца
        print("Not Уфа")
        
        if today_month == 1:
            findValues(country, city_id, city,today_year, 1, date_vals, day_temp_vals, evening_temp_vals, city_ids)
            findValues(country, city_id, city,today_year - 1, 12, date_vals, day_temp_vals, evening_temp_vals, city_ids)
        else:#take current and previous months
            
            months = [today_month, today_month-1]
            print("months", months)
            for month in months:
                findValues(country, city_id, city,today_year, month, date_vals, day_temp_vals, evening_temp_vals, city_ids)  
       
    df = pd.DataFrame({ 'CityId': np.array(city_ids),
                        'Date': np.array(date_vals),
                        'Day temperature': np.array(day_temp_vals),
                        'Evening temperature': np.array(evening_temp_vals)})
    print("start ", counter)
    print("end ", counter + len(df))
    df.index = np.arange(counter, counter + len(df))
    return df

In [8]:
import csv

In [27]:
cities = getCities('Россия')
print(cities)
cities.to_csv("cities.csv", sep=';', index=True, index_label='Id', encoding='utf-8')

                                                 Name
0        Москва, Москва (город федерального значения)
1   Санкт-Петербург, Санкт-Петербург (город федера...
2              Нижний Новгород, Нижегородская область
3                           Самара, Самарская область
4                      Челябинск, Челябинская область
..                                                ...
95                       Энгельс, Саратовская область
96                   Нижнекамск, Республика Татарстан
97                         Туапсе, Краснодарский край
98                    Великие Луки, Псковская область
99                          Шахты, Ростовская область

[100 rows x 1 columns]


In [28]:
cities_df = pd.read_csv("cities.csv", index_col=['Id'], sep=';')

In [29]:
cities_df

,Name
Id,
0,"Москва, Москва (город федерального значения)"
1,"Санкт-Петербург, Санкт-Петербург (город федера..."
2,"Нижний Новгород, Нижегородская область"
3,"Самара, Самарская область"
4,"Челябинск, Челябинская область"
...,...
95,"Энгельс, Саратовская область"
96,"Нижнекамск, Республика Татарстан"
97,"Туапсе, Краснодарский край"


In [30]:
df = pd.DataFrame(columns = ['CityId',
                        'Date',
                        'Day temperature',
                        'Evening temperature'])

In [31]:
df.to_csv("temp.csv", sep=';', index=True, index_label='Id', encoding='utf-8')

In [37]:
def getAllData(df): #get all values for all cities and save to df
    country = 'Россия'
    names = cities_df["Name"].to_numpy()    
    ids = cities_df.index.values
    #print("names: ", names)
    #print("ids: ", ids)
    counter = 0 #to support indices (last index from csv + 1)
    for i in range(names.size):
        city = names[i]
        city_id = ids[i]
        print("City: ", city)
        print("city_id: ", city_id)
        new_df = getWeather(country, city, city_id, counter, 'Уфа, Республика Башкортостан')
        new_df.to_csv("temp.csv", mode='a', sep=';', encoding='utf-8', header=False,index=True, index_label='Id')
        print("Added city ", city)
        counter += len(new_df)
        print(counter)
    return df, cities

In [38]:
tmps, cities = getAllData(df)

City:  Москва, Москва (город федерального значения)
city_id:  0
today_year =  2023
today_month =  3
Not Уфа
months [3, 2]
Set country =  Россия
Set region =  Все области
Try to set city  Москва, Москва (город федерального значения)
Set city =  Москва, Москва (город федерального значения)
Set year =  2023
Set month =  Март
Set country =  Россия
Set region =  Все области
Try to set city  Москва, Москва (город федерального значения)
Set city =  Москва, Москва (город федерального значения)
Set year =  2023
Set month =  Февраль
start  0
end  35
Added city  Москва, Москва (город федерального значения)
35
City:  Санкт-Петербург, Санкт-Петербург (город федерального значения)
city_id:  1
today_year =  2023
today_month =  3
Not Уфа
months [3, 2]
Set country =  Россия
Set region =  Все области
Try to set city  Москва, Москва (город федерального значения)
Try to set city  Санкт-Петербург, Санкт-Петербург (город федерального значения)
Set city =  Санкт-Петербург, Санкт-Петербург (город федерального

Set month =  Март
Set country =  Россия
Set region =  Все области
Try to set city  Москва, Москва (город федерального значения)
Try to set city  Санкт-Петербург, Санкт-Петербург (город федерального значения)
Try to set city  Нижний Новгород, Нижегородская область
Try to set city  Самара, Самарская область
Try to set city  Челябинск, Челябинская область
Try to set city  Краснодар, Краснодарский край
Try to set city  Екатеринбург, Свердловская область
Try to set city  Уфа, Республика Башкортостан
Set city =  Уфа, Республика Башкортостан
Set year =  2022
Set month =  Март
Set country =  Россия
Set region =  Все области
Try to set city  Москва, Москва (город федерального значения)
Try to set city  Санкт-Петербург, Санкт-Петербург (город федерального значения)
Try to set city  Нижний Новгород, Нижегородская область
Try to set city  Самара, Самарская область
Try to set city  Челябинск, Челябинская область
Try to set city  Краснодар, Краснодарский край
Try to set city  Екатеринбург, Свердловс

Try to set city  Ростов-на-Дону, Ростовская область
Set city =  Ростов-на-Дону, Ростовская область
Set year =  2023
Set month =  Февраль
start  655
end  690
Added city  Ростов-на-Дону, Ростовская область
690
City:  Саратов, Саратовская область
city_id:  10
today_year =  2023
today_month =  3
Not Уфа
months [3, 2]
Set country =  Россия
Set region =  Все области
Try to set city  Москва, Москва (город федерального значения)
Try to set city  Санкт-Петербург, Санкт-Петербург (город федерального значения)
Try to set city  Нижний Новгород, Нижегородская область
Try to set city  Самара, Самарская область
Try to set city  Челябинск, Челябинская область
Try to set city  Краснодар, Краснодарский край
Try to set city  Екатеринбург, Свердловская область
Try to set city  Уфа, Республика Башкортостан
Try to set city  Пермь, Пермский край
Try to set city  Ростов-на-Дону, Ростовская область
Try to set city  Саратов, Саратовская область
Set city =  Саратов, Саратовская область
Set year =  2023
Set month

Try to set city  Краснодар, Краснодарский край
Try to set city  Екатеринбург, Свердловская область
Try to set city  Уфа, Республика Башкортостан
Try to set city  Пермь, Пермский край
Try to set city  Ростов-на-Дону, Ростовская область
Try to set city  Саратов, Саратовская область
Try to set city  Омск, Омская область
Try to set city  Новосибирск, Новосибирская область
Try to set city  Воронеж, Воронежская область
Try to set city  Волгоград, Волгоградская область
Set city =  Волгоград, Волгоградская область
Set year =  2023
Set month =  Февраль
start  832
end  867
Added city  Волгоград, Волгоградская область
867
City:  Барнаул, Алтайский край
city_id:  15
today_year =  2023
today_month =  3
Not Уфа
months [3, 2]
Set country =  Россия
Set region =  Все области
Try to set city  Москва, Москва (город федерального значения)
Try to set city  Санкт-Петербург, Санкт-Петербург (город федерального значения)
Try to set city  Нижний Новгород, Нижегородская область
Try to set city  Самара, Самарска

Try to set city  Екатеринбург, Свердловская область
Try to set city  Уфа, Республика Башкортостан
Try to set city  Пермь, Пермский край
Try to set city  Ростов-на-Дону, Ростовская область
Try to set city  Саратов, Саратовская область
Try to set city  Омск, Омская область
Try to set city  Новосибирск, Новосибирская область
Try to set city  Воронеж, Воронежская область
Try to set city  Волгоград, Волгоградская область
Try to set city  Барнаул, Алтайский край
Try to set city  Тюмень, Тюменская область
Try to set city  Казань, Республика Татарстан
Try to set city  Белгород, Белгородская область
Set city =  Белгород, Белгородская область
Set year =  2023
Set month =  Февраль
start  974
end  1009
Added city  Белгород, Белгородская область
1009
City:  Красноярск, Красноярский край
city_id:  19
today_year =  2023
today_month =  3
Not Уфа
months [3, 2]
Set country =  Россия
Set region =  Все области
Try to set city  Москва, Москва (город федерального значения)
Try to set city  Санкт-Петербург, 

...

Try to set city  Ульяновск, Ульяновская область
Try to set city  Кемерово, Кемеровская область
Try to set city  Иркутск, Иркутская область
Try to set city  Чебоксары, Чувашская республика
Try to set city  Сургут, Ханты-Мансийский автономный округ - Югра
Try to set city  Липецк, Липецкая область
Try to set city  Набережные Челны, Республика Татарстан
Try to set city  Тула, Тульская область
Try to set city  Брянск, Брянская область
Try to set city  Тамбов, Тамбовская область
Try to set city  Ставрополь, Ставропольский край
Try to set city  Рязань, Рязанская область
Try to set city  Курск, Курская область
Try to set city  Ярославль, Ярославская область
Try to set city  Нижневартовск, Ханты-Мансийский автономный округ - Югра
Try to set city  Смоленск, Смоленская область
Try to set city  Магнитогорск, Челябинская область
Try to set city  Нижний Тагил, Свердловская область
Try to set city  Тверь, Тверская область
Try to set city  Воркута (а/п), Республика Коми
Try to set city  Петрозаводск, 

Set country =  Россия
Set region =  Все области
Try to set city  Москва, Москва (город федерального значения)
Try to set city  Санкт-Петербург, Санкт-Петербург (город федерального значения)
Try to set city  Нижний Новгород, Нижегородская область
Try to set city  Самара, Самарская область
Try to set city  Челябинск, Челябинская область
Try to set city  Краснодар, Краснодарский край
Try to set city  Екатеринбург, Свердловская область
Try to set city  Уфа, Республика Башкортостан
Try to set city  Пермь, Пермский край
Try to set city  Ростов-на-Дону, Ростовская область
Try to set city  Саратов, Саратовская область
Try to set city  Омск, Омская область
Try to set city  Новосибирск, Новосибирская область
Try to set city  Воронеж, Воронежская область
Try to set city  Волгоград, Волгоградская область
Try to set city  Барнаул, Алтайский край
Try to set city  Тюмень, Тюменская область
Try to set city  Казань, Республика Татарстан
Try to set city  Белгород, Белгородская область
Try to set city  К

Try to set city  Хабаровск, Хабаровский край
Try to set city  Анапа, Краснодарский край
Try to set city  Владивосток, Приморский край
Try to set city  Старый Оскол, Белгородская область
Try to set city  Геленджик, Краснодарский край
Try to set city  Нефтеюганск, Ханты-Мансийский автономный округ - Югра
Try to set city  Орск, Оренбургская область
Try to set city  Калуга, Калужская область
Try to set city  Стерлитамак, Республика Башкортостан
Try to set city  Ухта, Республика Коми
Try to set city  Волжский, Волгоградская область
Try to set city  Кострома, Костромская область
Try to set city  Бийск, Алтайский край
Try to set city  Златоуст, Челябинская область
Try to set city  Альметьевск, Республика Татарстан
Try to set city  Ишим, Тюменская область
Try to set city  Волгодонск, Ростовская область
Try to set city  Братск, Иркутская область
Try to set city  Ханты-Мансийск, Ханты-Мансийский автономный округ - Югра
Try to set city  Новый Уренгой, Ямало-Ненецкий автономный округ
Try to set ci

Try to set city  Липецк, Липецкая область
Try to set city  Набережные Челны, Республика Татарстан
Try to set city  Тула, Тульская область
Try to set city  Брянск, Брянская область
Try to set city  Тамбов, Тамбовская область
Try to set city  Ставрополь, Ставропольский край
Try to set city  Рязань, Рязанская область
Try to set city  Курск, Курская область
Try to set city  Ярославль, Ярославская область
Try to set city  Нижневартовск, Ханты-Мансийский автономный округ - Югра
Try to set city  Смоленск, Смоленская область
Try to set city  Магнитогорск, Челябинская область
Try to set city  Нижний Тагил, Свердловская область
Try to set city  Тверь, Тверская область
Try to set city  Воркута (а/п), Республика Коми
Try to set city  Петрозаводск, Республика Карелия
Try to set city  Сочи, Краснодарский край
Try to set city  Орел, Орловская область
Try to set city  Новороссийск, Краснодарский край
Try to set city  Вологда, Вологодская область
Try to set city  Таганрог, Ростовская область
Try to set

Set country =  Россия
Set region =  Все области
Try to set city  Москва, Москва (город федерального значения)
Try to set city  Санкт-Петербург, Санкт-Петербург (город федерального значения)
Try to set city  Нижний Новгород, Нижегородская область
Try to set city  Самара, Самарская область
Try to set city  Челябинск, Челябинская область
Try to set city  Краснодар, Краснодарский край
Try to set city  Екатеринбург, Свердловская область
Try to set city  Уфа, Республика Башкортостан
Try to set city  Пермь, Пермский край
Try to set city  Ростов-на-Дону, Ростовская область
Try to set city  Саратов, Саратовская область
Try to set city  Омск, Омская область
Try to set city  Новосибирск, Новосибирская область
Try to set city  Воронеж, Воронежская область
Try to set city  Волгоград, Волгоградская область
Try to set city  Барнаул, Алтайский край
Try to set city  Тюмень, Тюменская область
Try to set city  Казань, Республика Татарстан
Try to set city  Белгород, Белгородская область
Try to set city  К

In [18]:
import argparse
import sqlite3
import shutil
import time
import os

In [19]:
def insert_values_to_table(table_name, cursor, file_name, conn): #функция, в которой берем из файла значения и вставляем в таблицу при помощи to_sql
    df = pd.read_csv(file_name, sep=";")
  #print(df)
    df.columns = get_column_names_from_db_table(cursor, table_name)
    df.to_sql(name=table_name, con=conn, if_exists='append', index=False)    

def get_column_names_from_db_table(cursor, table_name): #функция, в которой берем наименования колонок
   

    table_column_names = 'PRAGMA table_info(' + table_name + ');'
    cursor.execute(table_column_names)
    table_column_names = cursor.fetchall()

    column_names = list()

    for name in table_column_names:
        column_names.append(name[1])
    #print("columns:", column_names)
    return column_names

In [20]:
conn = sqlite3.connect('weather.db', timeout=20) 
c = conn.cursor()

In [21]:
c.execute('''
          CREATE TABLE IF NOT EXISTS cities
          ([id] INTEGER PRIMARY KEY, [name] TEXT)
          ''')

In [22]:
c.execute('''
          CREATE TABLE IF NOT EXISTS temperature
          ([id] INTEGER PRIMARY KEY, INTEGER REFERENCES cities (id), [date] TEXT, [day] TEXT, [evening] TEXT)
          ''')

In [39]:
insert_values_to_table('cities', c, 'cities.csv', conn) 

In [40]:
insert_values_to_table('temperature', c, 'temp.csv', conn) 

In [41]:
c.execute("SELECT * FROM cities;")
cities = pd.DataFrame(c.fetchall(), columns=['id', 'name'])
cities

,id,name
0,0,"Москва, Москва (город федерального значения)"
1,1,"Санкт-Петербург, Санкт-Петербург (город федера..."
2,2,"Нижний Новгород, Нижегородская область"
3,3,"Самара, Самарская область"
4,4,"Челябинск, Челябинская область"
...,...,...
95,95,"Энгельс, Саратовская область"
96,96,"Нижнекамск, Республика Татарстан"
97,97,"Туапсе, Краснодарский край"
98,98,"Великие Луки, Псковская область"


In [43]:
c.execute("SELECT * FROM temperature;")
temperatures = pd.DataFrame(c.fetchall(), columns=['id', 'city_id', 'date', 'day', 'evening'])
temperatures

,id,city_id,date,day,evening
0,0,0,2023/3/1,-2.0,-4.0
1,1,0,2023/3/2,-2.0,-2.0
2,2,0,2023/3/3,1.0,0.0
3,3,0,2023/3/4,1.0,0.0
4,4,0,2023/3/5,-1.0,-3.0
...,...,...,...,...,...
3868,3868,99,2023/2/24,-1.0,-5.0
3869,3869,99,2023/2/25,3.0,2.0
3870,3870,99,2023/2/26,10.0,8.0
3871,3871,99,2023/2/27,13.0,9.0
